In [1]:
import pandas as pd

from src.spacy_models.spacy_pipe_process import SpacyPipeProcess
from src.spacy_models.spacy_utils import SpacyExt, SpacyTask
from src.settings.enums import NaturalLanguage
from src.data.data_loader import DataLoader

In [2]:
nat_lang = NaturalLanguage.DE

In [3]:
coref = SpacyPipeProcess(natural_language=nat_lang, spacy_task=SpacyTask.COREF, use_gpu=True)

GPU is used: True
custom extensions "init_extensions" initialized
regex_entity_pattern for own_regex_search were compiled.
Function "own_regex_search" initialized
Function "own_coref_resolve" initialized


In [4]:
coref.nlp.pipe_names

['tok2vec', 'senter', 'own_regex_search', 'llm_coref_resolve']

In [5]:
from src.settings.params import abbrevs_and_company_suffixes_with_dot_at_end
sorted(abbrevs_and_company_suffixes_with_dot_at_end)

['a.o.',
 'a.rh.',
 'a.s.',
 'ag & co.',
 'amtl.',
 'anh.',
 'ank.',
 'anl.',
 'anm.',
 'anschl.',
 'app.',
 'art.',
 'aufl.',
 'ausg.',
 'b.a.',
 'b.s.',
 'b.v.',
 'b.w.',
 'bd.',
 'bde.',
 'beil.',
 'bes.',
 'betr.',
 'bev.',
 'bez.',
 'bhf.',
 'brit.',
 'bspw.',
 'bzgl.',
 'bzw.',
 'c.p.a.',
 'chr.',
 'co.',
 'corp.',
 'd.ae.',
 'd.h.',
 'd.i.',
 'd.j.',
 'd.m.',
 'd.o.',
 'dazw.',
 'de.',
 'desgl.',
 'dgl.',
 'dipl.',
 'dir.',
 'dr.',
 'dt.',
 'dtzd.',
 'e.h.',
 'e.v.',
 'e.wz.',
 'ehem.',
 'eigtl.',
 'einschl.',
 'engl.',
 'entspr.',
 'erb.',
 'erw.',
 'ev.',
 'evtl.',
 'exkl..',
 'fa.',
 'fam.',
 'ffm.',
 'fr.',
 'frfr.',
 'frl.',
 'frz.',
 'geb.',
 'gebr.',
 'gedr.',
 'gegr.',
 'gek.',
 'ges.',
 'gesch.',
 'geschl.',
 'geschr.',
 'gest.',
 'gez.',
 'ggf.',
 'ggfs.',
 'gmbh & co.',
 'hpts.',
 'hptst.',
 'hr.',
 'hrn.',
 'hrsg.',
 'i.a.',
 'i.b.',
 'i.d.',
 'i.h.',
 'i.j.',
 'i.r.',
 'i.v.',
 'inc.',
 'ing.',
 'inh.',
 'inkl.',
 'inzw.',
 'j.d.',
 'jew.',
 'jh.',
 'jhrl.',
 'k.a.'

In [6]:
# coref.add_special_cases_to_tokenizer(special_cases=['ltd.'])
coref.set_custom_tokenizer(special_tokens=abbrevs_and_company_suffixes_with_dot_at_end)

In [7]:
dl = DataLoader()
year = 2023
month = 5
df = dl.load_monthly_df(year=year, month=month)

In [8]:
# without findings: 10, 11
index = 105
isin = df.loc[index, 'art_isin']
company = df.loc[index, 'art_company_name']
print('ISIN:', isin, '---', 'COMP:', company)
print('-----------------------------------')
text = df.loc[index, 'pp_art_text']
text

ISIN: DE0007571424 --- COMP: GK Software SE
-----------------------------------


'GK Software SE: Abschluss eines Delisting-Vertrages mit Fujitsu Delisting-Erwerbsangebot von Fujitsu zu EUR 190,00 je Aktie angekuendigt.  GK Software SE: Abschluss eines Delisting-Vertrages mit Fujitsu  Delisting-Erwerbsangebot von Fujitsu zu EUR 190,00 je Aktie angekuendigt.  Veroeffentlichung einer Insiderinformation nach Artikel 17 der Verordnung  Nr. 596 2014, uebermittelt durch EQS News - ein Service der EQS Group AG.  Die GK Software SE hat heute einen Delisting-Vertrag mit der Fujitsu Ltd. sowie mit deren 100%iger Tochtergesellschaft, der Fujitsu ND Solutions AG, abgeschlossen. Auf Grundlage dieses Vertrages soll die Stellung eines Antrags auf Widerruf der Zulassung der GK-Aktien am regulierten Markt erfolgen; zudem sollen wirtschaftlich angemessene Massnahmen getroffen werden, die erforderlich und fuer die Gesellschaft moeglich sind, um die Einbeziehung der GK-Aktien in den Handel im Freiverkehr zu beenden. Gemaess den Bestimmungen des Delisting-Vertrages wird die Bieterin he

In [9]:
%%time
doc = coref.nlp(text=text)

##### ->>>>>>>>>> COREF: PipeFunc.CUST_EXT_VAL_WAS_SET: True
ents_with_cust_exts: [{'start_char': 0, 'end_char': 11, 'ent_text': 'GK Software', 'comp_name': 'GK Software SE', 'comp_symbol': 'GKS.HM', 'set_in': 'own_regex_search'}, {'start_char': 139, 'end_char': 150, 'ent_text': 'GK Software', 'comp_name': 'GK Software SE', 'comp_symbol': 'GKS.HM', 'set_in': 'own_regex_search'}, {'start_char': 433, 'end_char': 447, 'ent_text': 'GK Software SE', 'comp_name': 'GK Software SE', 'comp_symbol': 'GKS.HM', 'set_in': 'own_regex_search'}, {'start_char': 2962, 'end_char': 2973, 'ent_text': 'GK Software', 'comp_name': 'GK Software SE', 'comp_symbol': 'GKS.HM', 'set_in': 'own_regex_search'}]
CPU times: user 2.53 s, sys: 147 ms, total: 2.68 s
Wall time: 2.68 s


In [ ]:
for sent in doc.sents:
    print(sent)
    print('----------------')

In [ ]:
coref.get_sentences_with_custom_extensions(processed_doc=doc)

In [ ]:
SpacyPipeProcess.get_ents_with_custom_extension(ents=doc.ents)

In [ ]:
# coref.nlp.tokenizer.explain('Die GK Software SE hat heute einen Delisting-Vertrag mit der Fujitsu Ltd. sowie mit deren 100%iger Tochtergesellschaft, der Fujitsu ND Solutions AG, abgeschlossen.')

In [ ]:
from src.settings.params import abbrevs_and_company_suffixes_with_dot_at_end
# coref.add_special_cases_to_tokenizer(special_cases=['Ltd.'])

In [ ]:
# coref.nlp.tokenizer.explain('Ltd.')

In [ ]:
# for sent in doc.sents:
#     if sent.ents:
#         ents_with_cust_ext = [{'start_char':e.start_char, 'end_char':e.end_char, 'ent_text':e.text, 'comp_name':getattr(e._, SpacyExt.COMP_NAME.ext_name), 'comp_symbol':getattr(e._, SpacyExt.COMP_SYMBOL.ext_name), 'set_in': getattr(e._, SpacyExt.SET_IN.ext_name)} for e in sent.ents if getattr(e._, SpacyExt.COMP_NAME.ext_name) != '']
#         s = {'sentence': sent.text, 'entities': ents_with_cust_ext} 
#         print(s)
#         print('----------------')
#     else:
#         continue
#     print('###############  SENTENCE END ####################')

In [ ]:
doc.text[2811:2817]